In [ ]:
import os
os.environ["GEMINI_API_KEY"] = "Your_GEMINI_API_KEY_Here"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Your_HUGGINGFACEHUB_API_TOKEN_Here"

In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

In [6]:
pip install langchain-google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.55.0
    Uninstalling google-genai-1.55.0:
      Successfully uninstalled google-genai-1.55.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, b

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## Step 1a - Indexing (Document Ingestion)

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

video_id = "Gfr50f6ZBvo"

try:
    api = YouTubeTranscriptApi()
    transcript_list = api.fetch(video_id, languages=["en"])

    # Use attribute access instead of dict indexing
    transcript = " ".join(chunk.text for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")


the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough 

In [5]:
 for s in transcript_list.snippets:
        print(f"[{s.start:6.2f}s → {s.start + s.duration:6.2f}s] {s.text}")

[  0.08s →   3.52s] the following is a conversation with
[  1.76s →   6.72s] demus hasabis
[  3.52s →   8.64s] ceo and co-founder of deepmind
[  6.72s →  11.20s] a company that has published and builds
[  8.64s →  13.20s] some of the most incredible artificial
[ 11.20s →  16.00s] intelligence systems in the history of
[ 13.20s →  16.88s] computing including alfred zero that
[ 16.00s →  18.96s] learned
[ 16.88s →  21.44s] all by itself to play the game of gold
[ 18.96s →  24.56s] better than any human in the world and
[ 21.44s →  25.68s] alpha fold two that solved protein
[ 24.56s →  28.72s] folding
[ 25.68s →  31.12s] both tasks considered nearly impossible
[ 28.72s →  33.12s] for a very long time
[ 31.12s →  35.60s] demus is widely considered to be one of
[ 33.12s →  37.12s] the most brilliant and impactful humans
[ 35.60s →  39.84s] in the history of artificial
[ 37.12s →  41.12s] intelligence and science and engineering
[ 39.84s →  43.76s] in general
[ 41.12s →  46.08s] this was tru

## Step 1b - Indexing (Text Splitting)

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [7]:
len(chunks)

168

In [8]:
chunks[0]

Document(metadata={}, page_content="the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to inter

## Step 1c- Indexing (Embedding Generation and Storing in Vector Store)*italicized text*

In [9]:
pip install langchain_huggingface

In [10]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings
embeddings = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

## Step 1d - Indexing (Embedding Generation and Storing in Vector Store)*italicized text*

In [11]:
vector_store = FAISS.from_documents(chunks, embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
vector_store.index_to_docstore_id

{0: '5493928b-58b9-46fb-9b78-b54becd2037f',
 1: 'bc2a11e8-064e-4cf7-91f3-18587bd205d9',
 2: 'cbb3371a-1da0-4c02-99b9-65419d377d6f',
 3: '63dc3aff-758d-4686-8939-d599535496cb',
 4: '424baaa6-8632-46a3-b924-cb5ed3100743',
 5: '649ce661-353a-47a8-9aad-06bc881b3482',
 6: '581982e9-2f49-4693-b890-dd93a56545b0',
 7: 'b630ab64-1c3c-4d79-b247-cc02bc521696',
 8: 'e93175ff-abf3-47e4-a0a0-4508ac00eb64',
 9: '9145fc2b-abba-4ac3-a1fc-0fe1b5d43df1',
 10: '06605605-3260-4bff-a248-60b1fe620dea',
 11: '6120d962-02df-4f78-972d-871b3c265038',
 12: '7d16e53d-6653-4623-9275-18756dac52d4',
 13: '94b8f3f7-30da-4a65-b4e8-4d6dcf0aa275',
 14: '7aa2ff12-975f-441d-be79-8b10eb716eff',
 15: '628286d5-05df-435d-b7d4-42f85c7e26e7',
 16: '3306a925-4a6c-4ceb-8a68-1f2b4b3b80ad',
 17: '2df59eaa-e1be-47a8-81c5-a18e1bc791f1',
 18: 'f5e5828b-e943-429b-b7d6-3553f13ee668',
 19: '3d4a87fd-7d30-470e-b08c-889bb681250e',
 20: '64bfd361-1529-494d-ae89-d95dc4fd448e',
 21: '666b7aa3-e0a7-4cef-a5ca-1696a73b78cc',
 22: '01775d16-5e85-

## Step 2 - Retrieval

In [13]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEndpointEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x780dd879c650>, search_kwargs={'k': 4})

In [14]:
retriever.invoke('What is deepmind')

[Document(id='96669937-f000-489d-8101-57a6dd4578a8', metadata={}, page_content="and how it works this is tough to uh ask you this question because you probably will say it's everything but let's let's try let's try to think to this because you're in a very interesting position where deepmind is the place of some of the most uh brilliant ideas in the history of ai but it's also a place of brilliant engineering so how much of solving intelligence this big goal for deepmind how much of it is science how much is engineering so how much is the algorithms how much is the data how much is the hardware compute infrastructure how much is it the software computer infrastructure yeah um what else is there how much is the human infrastructure and like just the humans interact in certain kinds of ways in all the space of all those ideas how much does maybe like philosophy how much what's the key if um uh if if you were to sort of look back like if we go forward 200 years look back what was the key 

## Step 3 - Augmentation

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")

In [16]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [17]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [18]:
retrieved_docs

[Document(id='7ada6d1a-0cb3-4ca6-9d85-9173ba734b81', metadata={}, page_content="in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be intere

In [19]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our point of view from an ai point of\n\

In [20]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [21]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we 

## Step 4 - Generation

In [22]:
answer = llm.invoke(final_prompt)
print(answer.content)

[{'type': 'text', 'text': 'Yes, the topic of nuclear fusion is discussed. According to the transcript:\n\n*   **Significance:** Fusion is described as a "transformative" area that could help accelerate progress in energy and climate.\n*   **Collaboration:** The team collaborated with EPFL (the Swiss Technical Institute) in Switzerland to use their test reactor for experiments.\n*   **Methodology:** They look at "bottleneck problems" from first principles and identify which ones are amenable to AI methods. \n*   **Achievements:** They published a paper in *Nature* titled "Magnetic control of tokamak plasmas to deep reinforcement learning." Using deep reinforcement learning (RL), they successfully controlled and held high-temperature plasma in specific shapes (such as droplets) for record amounts of time.\n*   **Challenges:** Fusion involves significant challenges in physics, material science, and engineering, particularly in building massive reactors and containing plasma.\n*   **Future

## Building a Chain

In [23]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [24]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [25]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [26]:
parallel_chain.invoke('who is Demis')

{'context': "to get world peace because there's also other corrupting things like wanting power over people and this kind of stuff which is not necessarily satisfied by by just abundance but i think it will help um and i think uh but i think ultimately ai is not going to be run by any one person or one organization i think it should belong to the world belong to humanity um and i think maybe many there'll be many ways this will happen and ultimately um everybody should have a say in that do you have advice for uh young people in high school and college maybe um if they're interested in ai or interested in having a big impact on the world what they should do to have a career they can be proud of her to have a life they can be proud of i love giving talks to the next generation what i say to them is actually two things i i think the most important things to learn about and to find out about when you're when you're young is what are your true passions is first of all there's two things on

In [27]:
parser = StrOutputParser()

In [28]:
main_chain = parallel_chain | prompt | llm | parser

In [29]:
main_chain.get_graph().print_ascii()

              +---------------------------------+           
              | Parallel<context,question>Input |           
              +---------------------------------+           
                    ****                ****                
                 ***                        ***             
               **                              ***          
+----------------------+                          **        
| VectorStoreRetriever |                           *        
+----------------------+                           *        
            *                                      *        
            *                                      *        
            *                                      *        
    +-------------+                         +-------------+ 
    | format_docs |                         | Passthrough | 
    +-------------+*                        +-------------+ 
                    ****                ****                
                        

In [30]:
main_chain.invoke('Can you summarize the video')

'Based on the transcript provided, the video is a conversation between Lex and Demis Hassabis. The summary of their discussion includes:\n\n*   **Explaining the Universe:** They discuss how a more fundamental and "simpler explanation" of the world might eventually replace the current standard model of physics and encompass mysteries like consciousness, life, and gravity.\n*   **Defining Intelligence:** Demis suggests that a sign of intelligence is the ability to explain complex topics simply, referencing Richard Feynman as an example.\n*   **Human-Computer Symbiosis:** They touch on how humans are already symbiotic with devices and the potential for enhancing ourselves.\n*   **AI and Chess History:** They discuss the history of AI in chess, mentioning Claude Shannon\'s 1949 program and Alan Turing, who had to run his own chess program by hand because computers were too slow. \n*   **Deep Blue vs. Kasparov:** They reflect on Deep Blue’s victory over Garry Kasparov. The speaker notes the